In [1]:
import pandas as pd
from tqdm import tqdm
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import os
import shutil
from functions import extractEanCodShopfarma, GeraDataframe, incluirPreco_PP
from password import passwordTrier

In [2]:
passwordTrier()

'0788'

In [3]:
dataframe = pd.read_excel('anticoncepcionais_28022023.xls')
#dataframe =dataframe.rename({'Reduz. ':'codshopfarma', 'Unnamed: 3': 'produto','Unnamed: 10':'ean'}, axis=1)
#dataframe.dropna(inplace=True)
dataframe

,codshopfarma,laboratorio,ean,produto,preco_venda,preco_compra,preco_pp,diferencapreco,%diff
0,112695,FQM GRUPO,7898040320683,"ADOLESS 0,060MG+0,015MG 28CP",37.99,28.33,34.53,3.46,9.107660
1,108423,FQM GRUPO,7898040322137,"ADOLESS 0,060MG+0,015MG 84CP",94.99,70.87,86.37,8.62,9.074639
2,13094,SANOFI,7896422532228,"AIXA 2MG+0,03MG 63CP REV",135.99,94.38,129.13,6.86,5.044489
3,14420,LEGRAND,7894916142809,ALGESTONA AC+ENAN EST INJ 1ML,10.99,9.11,11.12,-0.13,-1.182894
4,129781,ACHE,7896658035975,ALLESTRA 15 60MCG+15MCG 28CP R,20.99,16.15,19.81,1.18,5.621725
...,...,...,...,...,...,...,...,...,...
170,129802,BAYER SA,7891106914109,"YASMIN 3MG+0,03MG 63CP REV",255.99,176.58,243.44,12.55,4.902535
171,129801,BAYER SA,7891106914130,"YASMIN 3MG+0,03MG 63CP REV",255.99,197.37,243.44,12.55,4.902535
172,129803,BAYER SA,7891106914154,"YAZ 3MG+0,02MG 24+4CP REV",84.99,65.81,81.40,3.59,4.224026
173,10741,BAYER SA,7891106914147,"YAZ 3MG+0,02MG 24CP REV",84.99,58.87,85.91,-0.92,-1.082480


In [4]:
# Funçao para incluir observação de venda no cadastro dos produtos
def incluiObsVenda(dataframe, retira_obs=False):
    lst_error = []
    data =dataframe.dropna()
    #data = data.head(2)
    driver = webdriver.Chrome()
    driver.get("http://177.204.38.180:4647/sgfpod1/Cad_0020.pod")
    driver.implicitly_wait(15)
    sleep(1)
    driver.find_element(By.NAME, 'id_cod_usuario').send_keys(6)
    sleep(1)
    driver.find_element(By.NAME, 'nom_senha').send_keys(passwordTrier())
    sleep(1)
    driver.find_element(By.NAME, 'login').click()
    sleep(1)
    for index, row in tqdm(data.iterrows()):
        try:
            cod_shopfarma = int(row["codshopfarma"])
            driver.find_element(By.NAME, 'cod_redbarraEntrada').clear()
            driver.find_element(By.NAME, 'cod_redbarraEntrada').send_keys(cod_shopfarma)
            driver.find_element(By.NAME, 'cod_redbarraEntrada').send_keys(Keys.RETURN)
            sleep(1)
            if retira_obs:
                driver.find_element(By.LINK_TEXT, 'Configurações').click()
                sleep(1)
                driver.find_element(By.NAME, 'nom_obsvenda').clear()
                sleep(5)
                driver.refresh()
            else:
                driver.find_element(By.LINK_TEXT, 'Preço').click()
                valor_pmc = driver.find_element(By.NAME, 'vlr_pmc').get_attribute('value')
                valor_pmc = float(valor_pmc.replace(',','.'))
                sleep(1)
                valor_final = driver.find_element(By.NAME, 'vlr_venda').get_attribute('value')
                valor_final = float(valor_final.replace(',','.'))
                desconto = (valor_pmc - valor_final) / valor_pmc * 100
                sleep(1)
                driver.find_element(By.LINK_TEXT, 'Configurações').click()
                sleep(1)
                driver.find_element(By.NAME, 'nom_obsvenda').clear()
                sleep(5)
                driver.find_element(By.NAME, 'nom_obsvenda').send_keys('### CAMPANHA DOS ANTICONCEPCIONAIS ###')
                driver.find_element(By.NAME, 'nom_obsvenda').send_keys(Keys.CONTROL + Keys.RETURN)
                driver.find_element(By.NAME, 'nom_obsvenda').send_keys('OFERECA ESTES ITENS PARA SEU CLIENTE:')
                driver.find_element(By.NAME, 'nom_obsvenda').send_keys(Keys.CONTROL + Keys.RETURN)
                driver.find_element(By.NAME, 'nom_obsvenda').send_keys('IBUPRIL 400MG C/8')
                driver.find_element(By.NAME, 'nom_obsvenda').send_keys(Keys.CONTROL + Keys.RETURN)
                driver.find_element(By.NAME, 'nom_obsvenda').send_keys('KIT DERMAFEME SAB INTIMO')
                driver.find_element(By.NAME, 'nom_obsvenda').send_keys(Keys.CONTROL + Keys.RETURN)
                driver.find_element(By.NAME, 'nom_obsvenda').send_keys(f'O PMC deste produto é R$ {valor_pmc:.2f}. Desconto de {desconto:.2f}%')
                sleep(1)
                driver.find_element(By.NAME, 'salvar').click()
                sleep(3)
                print('Observação de venda incluida com sucesso. Código: ', cod_shopfarma)
                driver.refresh()
        except:
            print('Erro ao incluir observação de venda. Código: ', cod_shopfarma)
            lst_error.append(cod_shopfarma)
            driver.refresh()
            pass
    return data, lst_error

In [5]:
data, lst_error = incluiObsVenda(dataframe, retira_obs=False)
lst_error

0it [00:00, ?it/s]

Observação de venda incluida com sucesso. Código:  112695


1it [00:14, 14.58s/it]

Observação de venda incluida com sucesso. Código:  108423


2it [00:29, 14.58s/it]

...

Observação de venda incluida com sucesso. Código:  10741


174it [38:34, 14.54s/it]

Observação de venda incluida com sucesso. Código:  55566


175it [38:48, 13.31s/it]


[18719,
 16308,
 16929,
 18005,
 110781,
 89386,
 139415,
 14117,
 7444,
 72968,
 978,
 84666,
 13117,
 129799,
 120724,
 8140,
 104293,
 13329]

In [6]:
df_error = pd.DataFrame(lst_error, columns=['codshopfarma'])
df_error

,codshopfarma
0,18719
1,16308
2,16929
3,18005
4,110781
5,89386
6,139415
7,14117
8,7444
9,72968


In [ ]:
data_error, lst_error2 = incluiObsVenda(df_error)
data_error

In [ ]:
lst_error2